### Imports

In [ ]:
import pickle
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda, Conv2D, MaxPool2D, Reshape, Conv2DTranspose
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from keras import backend as K
from keras.models import load_model
import random
import matplotlib.pyplot as plt

### Load the training and testing data

In [3]:
with open('hw4_tr7.pkl', 'rb') as pickle_file:
    train_data = pickle.load(pickle_file)
print(train_data.shape)

(6265, 28, 28)


In [4]:
with open('hw4_te7.pkl', 'rb') as pickle_file:
    test_data = pickle.load(pickle_file)
print(test_data.shape)

(1028, 28, 28)


### Adding Channel Dimension

In [5]:
train_data = np.expand_dims(train_data, axis=3)

In [6]:
test_data = np.expand_dims(test_data, axis=3)

### Variational AutoEncoder

In [7]:
def VAE_model(ip_shape = [28, 28, 1], latent_dim = 4):
    input_img = Input(ip_shape) 
    
    encoder_model = Sequential()   
    encoder_model.add(Conv2D(16, kernel_size = (5, 5), input_shape = ip_shape, activation = 'relu'))
    encoder_model.add(Conv2D(32, kernel_size = (3, 3), activation = 'relu'))
    encoder_model.add(Conv2D(64, kernel_size = (2, 2), activation = 'relu'))
    encoder_model.add(Flatten())
    encoder_model.add(Dense(128))
    
    encoder_output = encoder_model(input_img)
    
    z_mean = Dense(latent_dim)(encoder_output)
    z_log_var = Dense(latent_dim)(encoder_output)
     
    epsilon =  K.random_normal(shape=K.shape(z_mean))
    #reparameterize
    z = z_mean + K.exp(0.5 + z_log_var) * epsilon
    
    decoder_model = Sequential()
    decoder_model.add(Dense(latent_dim*2, input_shape = z.get_shape(), activation = 'relu'))
    decoder_model.add(Dense(28*28, activation = 'relu'))
    decoder_model.add(Reshape(target_shape = (28,28, 1)))
    decoder_model.add(Conv2DTranspose(64, kernel_size = (2, 2), activation = 'relu', padding='SAME'))
    decoder_model.add(Conv2DTranspose(32, kernel_size = (3, 3), activation = 'relu', padding='SAME'))
    decoder_model.add(Conv2DTranspose(16, kernel_size = (5, 5), activation = 'relu', padding='SAME'))
    decoder_model.add(Conv2DTranspose(1, kernel_size = (5, 5), padding='SAME'))
    
    prediction = decoder_model(z)
      
    vae_model = Model(inputs = input_img, outputs = prediction)
    
    return vae_model, z_mean, z_log_var, decoder_model

### ELBO loss

In [8]:
def vae_loss(ip, op):
    kld = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    kld = kld / ( 28 * 28 )
    bce = tf.losses.sigmoid_cross_entropy(ip, op)
    
    return bce+kld

### Initialize and Compile Model

In [9]:
vae_model, z_mean, z_log_var, decoder_model = VAE_model() 
vae_model.compile(loss=vae_loss,optimizer = tf.keras.optimizers.Adam())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## Training

In [ ]:
vae_model.fit(train_data, train_data, batch_size=64, epochs=50, shuffle = True, validation_data = (test_data, test_data))

Train on 6265 samples, validate on 1028 samples
Epoch 1/50
6265/6265 [==============================] - 31s 5ms/sample - loss: 0.3265 - val_loss: 0.2396
Epoch 2/50
6265/6265 [==============================] - 27s 4ms/sample - loss: 0.2029 - val_loss: 0.1746
Epoch 3/50
4736/6265 [=====================>........] - ETA: 6s - loss: 0.1690

In [ ]:
vae_model.save_weights('./keras_checkpoints/vae.cpkt')

## Testing

In [ ]:
vae_model = model.load_weights('./keras_checkpoints/vae.cpkt')

## References
1. https://www.tensorflow.org/tutorials/generative/cvae
2. https://blog.keras.io/building-autoencoders-in-keras.html

In [ ]:
num_samples = 5

test_original = test_data[np.random.choice(test_data.shape[0], 5)]

test_recon = vae_model.predict(test_original)

In [ ]:
for i in range(num_samples):
  plt.figure(figsize=(5, 10))
  
  ax = plt.subplot(1, 2, 1)
  plt.imshow(test_original[i].reshape([28, 28]))
  ax.set_xticklabels([])
  ax.set_yticklabels([])
  ax.set_xticks([])
  ax.set_yticks([])
  
  ax = plt.subplot(1, 2, 2)
  plt.imshow(test_recon[i].reshape([28, 28]))
  ax.set_xticklabels([])
  ax.set_yticklabels([])
  ax.set_xticks([])
  ax.set_yticks([])

In [ ]:
vae_model.summary()

In [ ]:
latent_vector = vae_model.predict(train_data[0], output = vae_model.get_layer(index=11).output)

In [ ]:
intermediate_layer_model = Model(inputs=vae_model.input, outputs=vae_model.get_layer(index = 11).output)
intermediate_output = intermediate_layer_model.predict(np.expand_dims(test_original[0], 0))

In [ ]:
ref_img = intermediate_output

In [ ]:
def plot_images(images):
  n = len(images)
  plt.figure(figsize=(20, 20 * n))
  for i in range(n):
      ax = plt.subplot(1, n, i + 1)
      plt.imshow(images[i])
      ax.set_xticklabels([])
      ax.set_yticklabels([])
      ax.set_xticks([])
      ax.set_yticks([])

In [ ]:
ref_img

In [ ]:
num_tweaked_images = 10
for dim in range(4):
  tweaked_vectors = []
  for new_val in np.linspace(-2, 2, num_tweaked_images):
    tweaked_vec = ref_img.copy()
    tweaked_vec[0, dim] = new_val
    tweaked_vectors.append(tweaked_vec)
  gen_images = decoder_model.predict(np.array(tweaked_vectors))  
#   gen_images = sess.run(generate_image, feed_dict={Z: np.array(tweaked_vectors)})
  plot_images(gen_images.reshape([-1, 28, 28]))